In [ ]:
## Copy some codes from Google's example
## https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb
## About Download/load models and how to apply incoming images to model.

##Copy some codes from Dat Tran's example
## https://github.com/datitran/object_detector_app/blob/master/object_detection_app.py
## About setting parameters and how to apply pre-trained models to streaming videos.




import os
import cv2
import numpy as np
from picamera.array import PiRGBArray
from picamera import PiCamera
import tensorflow as tf
import argparse
import sys
from utils import label_map_util
from utils import visualization_utils as vis_util



sys.path.append('..')
ckpt = os.path.join(os.getcwd(),'ssdlite_mobilenet_v2_coco_2018_05_09','frozen_inference_graph.pb')
labels = os.path.join(os.getcwd(),'data','mscoco_label_map.pbtxt')
max_classifications = 100

label_map = label_map_util.load_labelmap(labels)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=max_classifications, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


#Initialize Tensorflow part
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(ckpt, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

#images that gonna be feed into the model, which is the images streaming from Picamera.
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

#Locate the classified class
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

#Returned Probability
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')

#Returned classes according to the highest probability
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')



#Initialize Opencv2 part.

frame_rate_calc = 1
freq = cv2.getTickFrequency()
font = cv2.FONT_HERSHEY_SIMPLEX

#Frame_size
Camera_width = 1280
Camera_height = 720

#Picamera only ,not the one with USB
if camera_type == 'picamera':
    camera = PiCamera()
    camera.resolution = (Camera_width,Camera_height)
    camera.framerate = 20
    rawCapture = PiRGBArray(camera, size=(Camera_width,Camera_height))
    rawCapture.truncate(0)

    for i in camera.capture_continuous(rawCapture, format="bgr",use_video_port=True):

        t1 = cv2.getTickCount()
        frame = np.copy(i.array)
        frame.setflags(write=1)
        frame_expanded = np.expand_dims(frame, axis=0)
         
        #Testing    
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: frame_expanded})

        vis_util.visualize_boxes_and_labels_on_image_array(frame,np.squeeze(boxes),np.squeeze(classes).astype(np.int32),np.squeeze(scores),category_index,use_normalized_coordinates=True,line_thickness=8,min_score_thresh=0.40)

        cv2.putText(frame,"FPS: {0:.2f}".format(frame_rate_calc),(30,50),font,1,(255,255,0),2,cv2.LINE_AA)
        cv2.imshow('Object detector', frame)
        t2 = cv2.getTickCount()
        time1 = (t2-t1)/freq
        frame_rate_calc = 1/time1

        if cv2.waitKey(1) == ord('q'):
            break

        rawCapture.truncate(0)

    camera.close()

cv2.destroyAllWindows()